# Разведочный анализ данных абитуриентов ИИК
### Дано: анонимизированная выгрузка данных о студентах, поступивших в ИИК.
### На выходе: Jupyter Notebook, иллюстрирующий выполненное задание, с графиками и пояснениями.
### Первичная обработка
1.	Сделать преобразование данных в графе «пол» - М=мужской, Ж=женский
2.	Вместо полей "информатика" и "физика" завести два поля: "3балл " и "3экз" - в одном балл по физике или информатике, в другом строка, в которой написан предмет, по которому сдан экзамен (физика или информатика).
3.	Баллы за индивидуальные достижения трансформировать в два булевых столбца: "олимпиада" и "медаль"
4.	"Школа" распарсить в поля "населённый пункт" и "школа", добиться как можно меньшего числа вариантов написания одной и той же школы
Можно п. 4 отложить на потом, если будет тягостно, сконцентрировавшись на анализе того, что хорошо парсится.
### Анализ датасета
Провести разведочный анализ данных с целью выявления неожиданных примеров корреляции признаков, желательно, касающихся выбора направления. Для основного анализа можно рассматривать только поступление на бюджет. Остальных можно вообще убрать из анализа.

Например, попробовать проанализировать:
#### •	Есть ли изменения в портрете поступающего на ИБАС (или ИИК в целом) со временем? (в распределении баллов, гендере и пр.)
#### •	Есть ли отличия студента, поступающего на ИБАС в сравнении с ПМИ или ФИИТ или ИВТ или ИИК в целом (школы, пропорции в баллах ЕГЭ, олимпиады, медали)?


## Обработка данных

In [169]:
import numpy as np
import pandas as pd
import re
import scipy.stats as sts
import matplotlib.pyplot as plt
import math
plt.style.use('ggplot')

In [170]:
file_path = 'priem16_23.csv'  
data_frame = pd.read_csv(file_path, delimiter=';', dtype={
    'Сумма баллов': 'float64',
    'р.я': 'float64',
    'инф': 'float64',
    'мат': 'float64',
    'физ': 'float64',
    'Сумма баллов за индивидуальные достижения': 'float64',
    'Год поступления': 'int64'
})
data_frame['Пол'].replace({'М': 'Мужской', 'Ж': 'Женский'}, inplace=True)
print(data_frame.head())

       Пол Нуждается в общежитии Иностранное гражданство  \
0  Мужской                   нет               Казахстан   
1  Мужской                    да                 Нигерия   
2  Мужской                    да               Казахстан   
3  Мужской                   нет               Казахстан   
4  Мужской                    да               Казахстан   

                                               Школа     Вид приема  \
0  "Интеллектуальная школа физико-математического...  Общий конкурс   
1  "Колледж Notre Dame" D'aArique Bietry, Кот Д'И...    По договору   
2  "Общеобразовательная средняя школа №43" отдела...  Общий конкурс   
3  "Средняя общеобразовательная школа №16" города...  Общий конкурс   
4  "Средняя общеобразовательная школа №7" города ...  Общий конкурс   

                                        Набор ОП  Сумма баллов   р.я  инф  \
0           10.03.01 Информационная безопасность         227.0  70.0  NaN   
1           10.03.01 Информационная безопасность         1

In [171]:
data_frame.insert(11, '3балл', [data_frame['инф'] if data_frame['инф'][i] > 0 else data_frame['физ'][i] for i in range(data_frame['физ'].size)] )
data_frame.insert(12, '3экз', ['Информатика' if number > 0 else 'Физика' for number in data_frame['инф']])
data_frame = data_frame.drop(columns=['инф', 'физ'])

print(data_frame.head())

       Пол Нуждается в общежитии Иностранное гражданство  \
0  Мужской                   нет               Казахстан   
1  Мужской                    да                 Нигерия   
2  Мужской                    да               Казахстан   
3  Мужской                   нет               Казахстан   
4  Мужской                    да               Казахстан   

                                               Школа     Вид приема  \
0  "Интеллектуальная школа физико-математического...  Общий конкурс   
1  "Колледж Notre Dame" D'aArique Bietry, Кот Д'И...    По договору   
2  "Общеобразовательная средняя школа №43" отдела...  Общий конкурс   
3  "Средняя общеобразовательная школа №16" города...  Общий конкурс   
4  "Средняя общеобразовательная школа №7" города ...  Общий конкурс   

                                        Набор ОП  Сумма баллов   р.я   мат  \
0           10.03.01 Информационная безопасность         227.0  70.0  96.0   
1           10.03.01 Информационная безопасность        

In [172]:
data_frame['Олимпиада'] = data_frame['Баллы за индивидуальные достижения'].str.contains(
    'олимпиад', case=False, na=False)
data_frame['Медаль'] = data_frame['Баллы за индивидуальные достижения'].str.contains(
    'с отличием', case=False, na=False)
data_frame = data_frame.drop(columns=['Баллы за индивидуальные достижения'])

print(data_frame.head())

       Пол Нуждается в общежитии Иностранное гражданство  \
0  Мужской                   нет               Казахстан   
1  Мужской                    да                 Нигерия   
2  Мужской                    да               Казахстан   
3  Мужской                   нет               Казахстан   
4  Мужской                    да               Казахстан   

                                               Школа     Вид приема  \
0  "Интеллектуальная школа физико-математического...  Общий конкурс   
1  "Колледж Notre Dame" D'aArique Bietry, Кот Д'И...    По договору   
2  "Общеобразовательная средняя школа №43" отдела...  Общий конкурс   
3  "Средняя общеобразовательная школа №16" города...  Общий конкурс   
4  "Средняя общеобразовательная школа №7" города ...  Общий конкурс   

                                        Набор ОП  Сумма баллов   р.я   мат  \
0           10.03.01 Информационная безопасность         227.0  70.0  96.0   
1           10.03.01 Информационная безопасность        

In [173]:
def split_school_location(school_info):
    match = re.search(
        r'(.+),\s*((?:г\.|с\.|пгт\.|ж/д_ст\.|пос\.)\s*[^,]+)', school_info, re.IGNORECASE)
    if match:
        school = match.group(1).strip()
        city = match.group(2).strip()
        return school, city
    return school_info, ""


data_frame[['Образовательное учереждение', 'Населенный пункт']] = data_frame['Школа'].apply(lambda x: pd.Series(split_school_location(x)))
data_frame = data_frame.drop(columns=['Школа'])
print(data_frame.head())

       Пол Нуждается в общежитии Иностранное гражданство     Вид приема  \
0  Мужской                   нет               Казахстан  Общий конкурс   
1  Мужской                    да                 Нигерия    По договору   
2  Мужской                    да               Казахстан  Общий конкурс   
3  Мужской                   нет               Казахстан  Общий конкурс   
4  Мужской                    да               Казахстан  Общий конкурс   

                                        Набор ОП  Сумма баллов   р.я   мат  \
0           10.03.01 Информационная безопасность         227.0  70.0  96.0   
1           10.03.01 Информационная безопасность         143.0  42.0  53.0   
2         11.03.04 Электроника и наноэлектроника         190.0  65.0  76.0   
3  09.03.01 Информатика и вычислительная техника         232.0  73.0  96.0   
4         11.03.04 Электроника и наноэлектроника         189.0  75.0  56.0   

  3балл    3экз  Сумма баллов за индивидуальные достижения  Год поступления  \
0